In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import argparse
import pickle as pkl
import time
from copy import deepcopy
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm

In [2]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from random import shuffle
import torch.nn.functional as F
from torchvision.transforms import v2
from torch import nn

c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, end_maxpool = False):
        super(ResidualBlock, self).__init__()
        if(downsample is not None):
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same',bias=False),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU(inplace=False),
                            nn.MaxPool2d(kernel_size=2, stride=2)
                            )  # Changed inplace to False
        else:
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same',bias=False),
                            nn.BatchNorm2d(out_channels),
                            nn.Hardtanh(min_val=0.0, max_val=1.0, inplace=False)
                            )
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1,bias=False),
                        nn.BatchNorm2d(out_channels),
                        nn.Hardtanh(min_val=0.0, max_val=1.0, inplace=False))  # Changed inplace to False
        self.downsample = downsample
        self.relu = nn.Hardtanh(min_val=0.0, max_val=1.0, inplace=False)  # Changed inplace to False
        self.out_channels = out_channels
        self.end_maxpool = end_maxpool

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out = out + residual
        if self.end_maxpool:
            out = F.relu(out, inplace=False)
        else:
            out = F.hardtanh(out, inplace=False, min_val=0.0, max_val=1.0)   # Use non-in-place ReLU
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 2, in_chanels = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_chanels, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU(inplace=False))
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2, end_maxpool = True)
        self.avgpool = nn.MaxPool2d(7, stride=1)
        self.fc = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1, end_maxpool = False):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=1, padding='same',bias=False),
                nn.BatchNorm2d(planes),
                nn.ReLU(inplace=False),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            if i == blocks-1 and end_maxpool:
                layers.append(block(self.inplanes, planes, end_maxpool = True))
            else:
                layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.hardtanh(x, min_val=0.0, max_val=1.0)
        x = self.fc2(x)
        return x

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, end_maxpool = False):
        super(ResidualBlock, self).__init__()
        if(downsample is not None):
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same'),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU(inplace=False),
                            nn.MaxPool2d(kernel_size=2, stride=2)
                            )  # Changed inplace to False
        else:
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same'),
                            nn.BatchNorm2d(out_channels),
                            nn.Hardtanh(min_val=0.0, max_val=1.0, inplace=False)
                            )
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(out_channels),
                        nn.Hardtanh(min_val=0.0, max_val=1.0, inplace=False))  # Changed inplace to False
        self.downsample = downsample
        self.relu = nn.Hardtanh(min_val=0.0, max_val=1.0, inplace=False)  # Changed inplace to False
        self.out_channels = out_channels
        self.end_maxpool = end_maxpool

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out = out + residual
        if self.end_maxpool:
            out = F.relu(out, inplace=False)
        else:
            out = F.hardtanh(out, inplace=False, min_val=0.0, max_val=1.0)   # Use non-in-place ReLU
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 2, in_channels = 5):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU(inplace=False))
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2, end_maxpool = True)
        self.avgpool = nn.MaxPool2d(7, stride=1)
        self.fc = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.1)

    def _make_layer(self, block, planes, blocks, stride=1, end_maxpool = False):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=1, padding='same'),
                nn.BatchNorm2d(planes),
                nn.ReLU(inplace=False),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            if i == blocks-1 and end_maxpool:
                layers.append(block(self.inplanes, planes, end_maxpool = True))
            else:
                layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.hardtanh(x,min_val=0, max_val=1)
        # x = self.dropout(x)
        x = self.fc2(x)
        return x# F.hardtanh(x)

In [5]:
# model_resnet = ResNet(ResidualBlock, [5, 6, 6, 4], in_channels = 6, num_classes=101).to("cuda")

In [6]:
model_resnet = ResNet(ResidualBlock, [5, 6, 6, 4], num_classes = 101, in_channels=6)

In [7]:
model_resnet = torch.load("best_resnet_nCaltech101_ReLU1_ReLUmaxpool_EST_FC2_corrected_exp.pt", weights_only=False)
model_resnet.eval().to("cuda")
print(model_resnet)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer0): Sequential(
    (0): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Hardtanh(min_val=0.0, max_val=1.0)
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Hardtanh(min_val=0.0, max_val=1.0)
      )
      (relu): Hardtanh(min_val=0.0, max_val=1.0)
    )
    (1): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_s

In [8]:
temp = torch.rand((1,6,224,224))
model_resnet(temp.to("cuda"))

tensor([[-2.0439, -2.5614,  1.0997, -3.3898,  0.7834,  1.5576, -0.8800,  0.6590,
         -1.5126,  1.0864,  0.1628, -1.8188,  0.2477, -1.4779, -0.5599, -1.8114,
         -0.3942, -0.7469,  0.4051, -0.7018, -3.4831, -1.8707, -2.9679, -1.3461,
         -3.1930, -2.1993, -0.6713,  1.3067, -5.1448, -1.8813,  1.5626,  0.7617,
          0.1341, -2.5766, -1.0187, -0.8682,  1.2351, -1.6846, -0.3800,  1.2141,
          1.6978, -1.5449, -1.5274,  2.7839, -1.1501, -1.1818, -1.6145,  0.2546,
         -0.9642,  0.5107,  1.0889, -2.7781, -0.5333,  0.8462,  1.5289,  2.1188,
         -1.2613, -1.6228, -2.2690,  0.5117,  0.2113,  0.8845,  0.3006,  0.7372,
          1.4158, -4.1189,  1.6858, -0.9459, -4.8902, -3.1649, -1.1440,  2.5721,
         -0.6321, -2.0204,  3.3315, -2.0412, -3.2814, -0.8243, -2.3256, -0.1700,
         -1.9981, -1.4495, -0.2801, -2.6389,  0.5912,  1.9115,  1.3035, -1.6153,
         -2.6138,  2.3971, -0.9295, -2.7658,  0.6637, -0.7801, -1.2309, -0.3919,
         -3.4303, -2.1354, -

In [9]:
from SNN.Check_ReLU1 import fuse_conv_and_bn
from SNN.Check_ReLU1 import SpikingConv2D
from SNN.Check_ReLU1 import MaxMinPool2D
from SNN.Check_ReLU1 import LayerSNN_ReLU1
from SNN.Check_ReLU1 import SpikingDense_positive_ReLU1, SpikingDense_pos_all, SpikingDense

In [10]:
class ResNet_ttfs(nn.Module):
    def __init__(self, model : ResNet, in_channels = 5):
        super(ResNet_ttfs, self).__init__()
        model.eval()
        robustness_params={
            'noise':0.0,
            'time_bits':0,
            'weight_bits': 0,
            'latency_quantiles':0.0
        }
        model.to('cuda')
        conv_fused = fuse_conv_and_bn(model.conv1[0], model.conv1[1], device = 'cuda')
        self.conv_first = SpikingConv2D(64, "temp1", device = 'cuda', padding=(3,3), stride=2, kernel_size=(7,7),robustness_params=robustness_params, kernels=conv_fused.weight.data, biases= conv_fused.bias.data)
        max_vect = torch.tensor([1]*in_channels)
        tmin, tmax, max_vect, scalar = self.conv_first.set_params(0.0,1.0,max_vect)
        self.tmin_post_pool = tmax
        self.pool = MaxMinPool2D(3, tmax.data,2,1).to("cuda")
        self.layer0SNN = LayerSNN_ReLU1(model.layer0, 64, 64, 5,device = 'cuda')
        tmax_prev = tmax
        tmin, tmax, max_vect, scalar = self.layer0SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer1SNN = LayerSNN_ReLU1(model.layer1, 64, 128, 6,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer1SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer2SNN = LayerSNN_ReLU1(model.layer2, 128, 256, 6,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer2SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer3SNN = LayerSNN_ReLU1(model.layer3, 256, 512, 4,device = 'cuda',end_maxpool=True)
        tmin, tmax, max_vect, scalar = self.layer3SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.pool2 = MaxMinPool2D(7, tmax.data,1,0).to("cuda")
        # temp = torch.ones((512,2,2))
        # max_vect = ((temp.T)*max_vect[:512]).T
        # max_vect = max_vect.view(-1)

        max_vect = (torch.ones([512, 1, 1]).to("cuda").T*max_vect).T.contiguous().view(512).to("cuda")

        weights = model.fc.weight.detach().clone()
        biases = model.fc.bias.detach().clone()
        self.layer_fc = SpikingDense_positive_ReLU1(256, 512,"test",robustness_params=robustness_params,device = 'cuda',weights=weights, biases=biases)
        
        tmin, tmax, max_vect, scalar = self.layer_fc.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer_fc2 = SpikingDense_pos_all(101,256, '',model.fc2.weight, model.fc2.bias,robustness_params=robustness_params,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer_fc2.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.tmin, self.tmax = tmin, tmax
        self.scalar = scalar
    
    def forward(self, tj):
        spike_sum, v_sqare_sum = 0,0.0

        x, spike, v  = self.conv_first(tj)

        spike_sum += spike
        v_sqare_sum +=v
        x, spike = self.pool(x)

        spike_sum += spike
        # print(x.shape)
        x = x
        x, spike1, v1 = self.layer0SNN(x)

        x, spike2, v2 = self.layer1SNN(x)

        x, spike3, v3 = self.layer2SNN(x)

        x, spike4, v4 = self.layer3SNN(x)

        # print(x[1].max(),x[1].min())
        x, spike5 = self.pool2(x)
        x = x.contiguous().view(x.size(0), -1)
        x, spike6, v5 = self.layer_fc(x)

        x, spike7, v6 = self.layer_fc2(x)

        spike_sum += spike1+spike2+spike3+spike4+spike5+spike6+spike7
        v_sqare_sum += v1+v2+v3+v4+v5+v6
        return x, spike_sum, v_sqare_sum

In [11]:
model_ttfs = ResNet_ttfs(model_resnet,6)

c:\D\time_to_spike\in_torch\SNN\Check_ReLU1.py:280: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_max = torch.tensor(t_min + self.B_n*max_V+eps_V, dtype=torch.float64, requires_grad=False)
c:\D\time_to_spike\in_torch\SNN\Check_ReLU1.py:311: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_max = torch.tensor(max(t_min + self.B_n*max_V+eps_V, minimal_t_max), dtype=torch.float64, requires_grad=False)
c:\D\time_to_spike\in_torch\SNN\Check_ReLU1.py:279: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_min = torch.tensor(t_min, d

In [12]:
temp = torch.rand((1,6,224,224))
temp_ttfs = 1 - temp
# print(model_ttfs.forward(temp_ttfs))
out, spike_sum, v_sqare_sum = model_ttfs.forward(temp_ttfs.to("cuda"))

print(spike_sum, v_sqare_sum)
print(((model_ttfs.tmax - out)*model_ttfs.scalar)[0,:101] - ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,101:])

tensor(10489619, device='cuda:0') tensor(780884.1250, device='cuda:0', grad_fn=<AddBackward0>)
tensor([-2.3047, -2.6129,  1.0238, -3.5191,  0.8542,  1.5927, -0.8449,  0.6999,
        -1.6516,  1.3530,  0.3256, -1.8877,  0.3147, -1.5017, -0.3977, -1.6101,
        -0.4186, -0.6821,  0.4235, -0.8002, -3.6488, -2.0146, -3.1218, -1.3300,
        -3.2068, -2.3535, -0.6922,  1.4187, -5.3874, -1.8285,  1.5525,  0.4871,
         0.1841, -2.7197, -1.1802, -0.8284,  1.2499, -1.8337, -0.2663,  1.1777,
         1.5968, -1.5436, -1.7056,  3.0666, -1.1822, -1.2031, -1.8047,  0.2087,
        -0.8320,  0.5121,  1.4110, -2.7447, -0.6277,  0.8506,  1.4823,  2.1850,
        -1.3260, -1.7684, -2.2366,  0.7277,  0.3308,  0.8747,  0.2011,  0.5919,
         1.4320, -3.9982,  1.8748, -1.0424, -5.0349, -3.3587, -1.1495,  2.7507,
        -0.4605, -2.1810,  3.4284, -2.1270, -3.4800, -0.8655, -2.5851, -0.1547,
        -2.3788, -1.4022, -0.3896, -2.6428,  0.7120,  1.9771,  1.2329, -1.5045,
        -2.6971,  2.4494,

In [13]:
model_resnet(temp.to("cuda"))

tensor([[-2.3083, -2.6151,  1.0211, -3.5246,  0.8565,  1.5948, -0.8491,  0.6985,
         -1.6531,  1.3559,  0.3292, -1.8883,  0.3204, -1.5038, -0.3959, -1.6078,
         -0.4187, -0.6859,  0.4219, -0.7995, -3.6475, -2.0176, -3.1252, -1.3330,
         -3.2106, -2.3568, -0.6938,  1.4213, -5.3913, -1.8258,  1.5492,  0.4836,
          0.1815, -2.7211, -1.1813, -0.8289,  1.2542, -1.8368, -0.2622,  1.1782,
          1.5958, -1.5410, -1.7070,  3.0704, -1.1848, -1.2054, -1.8063,  0.2080,
         -0.8305,  0.5123,  1.4179, -2.7414, -0.6261,  0.8560,  1.4841,  2.1888,
         -1.3257, -1.7683, -2.2408,  0.7288,  0.3290,  0.8740,  0.2032,  0.5950,
          1.4307, -3.9975,  1.8764, -1.0434, -5.0378, -3.3639, -1.1498,  2.7539,
         -0.4584, -2.1867,  3.4322, -2.1307, -3.4832, -0.8650, -2.5867, -0.1495,
         -2.3814, -1.4034, -0.3902, -2.6420,  0.7140,  1.9767,  1.2312, -1.5042,
         -2.6960,  2.4518, -0.9463, -2.9616,  0.4576, -0.9474, -1.0724, -0.2636,
         -3.4938, -2.1277, -

In [14]:
from torchvision.transforms import v2
torch.manual_seed(19)

transforms = v2.Compose([
    # v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomRotation(60),
    v2.ToDtype(torch.float32)
])


class NCaltech101ImageDataset(Dataset):
    def __init__(self, img_dir_file, transform=None, target_transform=None):
        self.images = np.load(img_dir_file + '_x.npy')
        self.labels = np.load(img_dir_file + '_y.npy')
        self.transform = transform
        self.transform = transform
        self.stage = 0

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        # if self.transform:
        #     image = self.transform(image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        label_temp = np.zeros((101,))
        label_temp[label] = 1
        if self.stage == 0:
            return self.transform(torch.tensor(image)), torch.tensor(label_temp)
        else:
            return torch.tensor(image), torch.tensor(label_temp)
    
    def set_stage(self, stage):
        self.stage = stage

In [15]:
from torch.utils.data import DataLoader
# training_data = NCarsImageDataset("./Datasety/nCars_train_EST_exp_", transform=transforms)
data = NCaltech101ImageDataset("./Datasety/Ncaltech101_EST_trilinear_corr", transform=transforms)


In [16]:
data.set_stage(1)
spike_acc = 0
MAE_sum = 0
MSE_sum = 0
TP = 0
for i in tqdm(range(len(data))):#10
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            temp_data, temp_label = deepcopy(data[i])
            temp_ttfs = (1 - temp_data).unsqueeze(0).to("cuda")
            temp_data = temp_data.to("cuda")
            # print(temp_data.shape)
            # print(model_ttfs.forward(temp_ttfs))
            out, spike_sum, v_sqare_sum = model_ttfs.forward(temp_ttfs)
            spike_acc+=spike_sum
            output = ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,:101] - ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,101:]
            output_true = model_resnet(temp_data.unsqueeze(0))
            MAE = torch.sum((output-output_true).abs())/101.0
            MSE = torch.sqrt(torch.sum((output-output_true)*(output-output_true)))
            MAE_sum += MAE
            MSE_sum += MSE
            TP += (torch.argmax(temp_label)==torch.argmax(output)) 
            del temp_data, temp_ttfs
    if i%100==0:
        torch.cuda.empty_cache()
print(spike_acc/len(data))
print("Mean MAE:", MAE_sum/len(data))
print("Mean MSE:", MSE_sum/(len(data)*101.0))
print("accuracy:",float(TP)/len(data))

100%|██████████| 8709/8709 [1:20:31<00:00,  1.80it/s]

tensor(10675561., device='cuda:0')
Mean MAE: tensor(0.0067, device='cuda:0')
Mean MSE: tensor(0.0008, device='cuda:0')
accuracy: 0.6217705821563899


In [17]:
print(TP)

tensor(5415, device='cuda:0')


In [18]:
i = 8000

with torch.no_grad():
    with torch.cuda.amp.autocast():
        temp_data, temp_label = deepcopy(data[i])
        temp_ttfs = (1 - temp_data).unsqueeze(0).to("cuda")
        temp_data = temp_data.to("cuda")
        # print(temp_data.shape)
        # print(model_ttfs.forward(temp_ttfs))
        out, spike_sum, v_sqare_sum = model_ttfs.forward(temp_ttfs)
        spike_acc+=spike_sum
        output = ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,:101] - ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,101:]
        output_true = model_resnet(temp_data.unsqueeze(0))
        MAE = torch.sum((output-output_true).abs())/101.0
        MSE = torch.sqrt(torch.sum((output-output_true)*(output-output_true)))
        MAE_sum += MAE
        MSE_sum += MSE
        TP += (torch.argmax(temp_label)==torch.argmax(output)) 
        print(torch.argmax(temp_label), torch.argmax(output))
        del temp_data, temp_ttfs

tensor(91) tensor(67, device='cuda:0')
